#  &#x1F4D1; **作业 2: 音位分类 (分类)**

学习目标：
* 数据预处理：从原始波形中提取MFCC特征
* 分类：使用预提取的MFCC特征执行逐帧音位分类
* 熟悉并提高pytorch训练技巧，熟悉pytorch模块
相关资料：
* Slides地址: https://docs.google.com/presentation/d/1v6HkBWiJb8WNDcJ9_-2kwVstxUWml87b9CnA16Gdoio/edit?usp=sharing
* Kaggle地址: https://www.kaggle.com/c/ml2022spring-hw2
* 相关课程视频资源也可在B站获取  

前置知识：
- 音位：

(phonetics 语音) 音位，音素（区分单词的最小语音单位，英语sip中的s和zip中的z是两个不同的音素）  

例如：Machine Learning → M AH SH IH N L ER N IH NG M M M AH AH SH SH IH IH IH N N N N ... Machine

- MFCC：  

在语音识别（Speech recognition）和话者识别（Speaker recognition）方面，最常用到的语音特征就是梅尔倒谱系数（Mel-scale Frequency Cepstral Coefficients, MFCC）。  

根据人耳听觉机理的研究发现，人耳对不同频率的声波有不同的听觉敏感度。从200Hz到5000Hz的语音信号对语音的清晰度影响对大。两个响度不等的声音作用于人耳时，则响度较高的频率成分的存在会影响到对响度较低的频率成分的感受，使其变得不易察觉，这种现象称为掩蔽效应。由于频率较低的声音在内耳蜗基底膜上行波传递的距离（速度）大于频率较高的声音，故一般来说，低音容易掩蔽高音，而高音掩蔽低音较困难。在低频处的声音掩蔽的临界带宽较高频要小。所以，人们从低频到高频这一段频带内按临界带宽的大小由密到疏安排一组带通滤波器，对输入信号进行滤波。将每个带通滤波器输出的信号能量作为信号的基本特征，对此特征经过进一步处理后就可以作为语音的输入特征。由于这种特征不依赖于信号的性质，对输入信号不做任何的假设和限制，又利用了听觉模型的研究成果。因此，这种参数比基于声道模型的LPCC相比具有更好的鲁棒性，更符合人耳的听觉特性，而且当信噪比降低时仍然具有较好的识别性能。  

在本次作业中，正常的一段音频素材可能包含大量的音位信息，而音位之间又可能存在重叠干扰的情况，因此我们将一段音频素材每隔10ms切取25ms，以此来尽可能保存完整的音位素材，取出后的素材称为一个frame，取出后的frame并不适合直接进入训练，因此我们要进行进一步的处理，通过MFCC，将它转化为一个39维度的特征，转换后为了更加精确的判断当前特征内的音位信息，我们往往采取其前后的特征来做辅助判断 也就是前向特征与后向特征各取5个，所以我们最后得到的是一个11*39维的一个向量。

![](./pic/01.png) 
想要深入了解实现过程的可以查看下列链接：  
[Prof. Hung-Yi Lee[2020Spring DLHLP] Speech Recognition](https://speech.ee.ntu.edu.tw/~tlkagk/courses/DLHLP20/ASR%20(v12).pdf)  
[ Prof. Lin-Shan Lee’s[Introduction to Digital Speech Processing]Chap.7](http://ocw.aca.ntu.edu.tw/ntu-ocw/ocw/cou/104S204)

#  数据集下载
如果下列命令无法下载，可以到下列地址下载数据
- Kaggle下载数据:  [Kaggle: ml2022spring-hw2](https://www.kaggle.com/competitions/ml2022spring-hw2)
- 百度云下载数据: [云盘(提取码：05zc)](https://pan.baidu.com/s/198xn8Lk9MjvUsq866mZuuw)


下载完成后，你应该能够获取如下文件：
- `libriphone/train_split.txt`
- `libriphone/train_labels`
- `libriphone/test_split.txt`
- `libriphone/feat/train/*.pt`: training feature<br>
- `libriphone/feat/test/*.pt`:  testing feature<br>  

pt文件可以使用torch.load方法导入


<b>同学们下载完后直接解压到 HW02文件夹下面（将里面的文件最终放到HW02下）</b>

In [1]:
# 下载链接
!wget -O libriphone.zip "https://github.com/xraychen/shiny-robot/releases/download/v1.0/libriphone.zip"

# 下列数据获取方式需要依靠gdown

# 备用链接 0
# !pip install --upgrade gdown
# !gdown --id '1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc' --output libriphone.zip

# 备用链接 1
# !pip install --upgrade gdown
# !gdown --id '1R1uQYi4QpX0tBfUWt2mbZcncdBsJkxeW' --output libriphone.zip

# 备用链接 2
# !wget -O libriphone.zip "https://www.dropbox.com/s/wqww8c5dbrl2ka9/libriphone.zip?dl=1"

# 备用链接 3
# !wget -O libriphone.zip "https://www.dropbox.com/s/p2ljbtb2bam13in/libriphone.zip?dl=1"

!unzip -q libriphone.zip
!ls libriphone

'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'unzip' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [2]:
# 输入如下指令查看GPU状态
!nvidia-smi

Sat Jun 29 16:29:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.33                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   42C    P8               2W /  50W |   1324MiB /  4096MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 准备数据

**Helper函数用于预处理来自每个话语的原始MFCC特征的训练数据**


一个音位可能跨越几个帧，并且取决于过去和将来的帧

因此，我们连接相邻的音位进行训练以获得更高的准确性。**concat_fatte**函数连接过去和未来的k帧（总共2k+1＝n帧），我们预测中心帧。


可以随意修改数据预处理函数，但**不要删除任何帧**（如果您修改函数，请记住检查帧的数量是否与幻灯片中提到的相同）

In [3]:
import os
import random
import pandas as pd
# 导入pytorch库，用于深度学习模型的训练和预测
import torch
# 导入进度条库，用于在数据加载和处理过程中显示进度
from tqdm import tqdm

# 定义导入feature的函数，用于加载特征文件
def load_feat(path):
    # 使用torch.load加载特征文件
    feat = torch.load(path)
    return feat

# 定义特征平移函数，用于对特征进行上下文扩展
def shift(x, n):
    # 如果n小于0，则向左平移
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    # 如果n大于0，则向右平移
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    # 如果n等于0，则不进行平移
    else:
        return x

    # 将平移后的特征拼接起来
    return torch.cat((left, right), dim=0)

# 定义特征拼接函数，用于将上下文特征拼接在一起
def concat_feat(x, concat_n):
    # 断言n为奇数，保证中心位置只有一个特征
    assert concat_n % 2 == 1  # n 必须是奇数
    # 如果n小于2，则直接返回原特征
    if concat_n < 2:
        return x
    # 获取序列长度和特征维度
    seq_len, feature_dim = x.size(0), x.size(1)
    # 将特征重复n次
    x = x.repeat(1, concat_n) 
    # 将特征转换为三维张量
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2)  # concat_n, seq_len, feature_dim
    # 计算中间位置
    mid = (concat_n // 2)
    # 对中间位置两侧的特征进行平移
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    # 将特征转换为二维张量
    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

# 定义数据预处理函数，用于加载数据集并进行预处理
def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1337):
    # 定义类别数量
    class_num = 41  # NOTE: 预先计算，不需要更改
    # 根据split参数确定数据集模式
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    # 定义标签字典，用于存储每个文件的标签
    label_dict = {}
    # 如果模式不为测试，则读取标签文件
    if mode != 'test':
        phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

        for line in phone_file:
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]

    # 如果split参数为训练或验证，则分割训练和验证数据
    if split == 'train' or split == 'val':
        # 读取训练分割文件
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        # 设置随机种子，保证数据分割的可重复性
        random.seed(train_val_seed)
        # 随机打乱数据列表
        random.shuffle(usage_list)
        # 根据train_ratio计算训练集和验证集的大小
        percent = int(len(usage_list) * train_ratio)
        # 分割训练集和验证集
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    # 如果split参数为测试，则读取测试数据列表
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        # 如果split参数无效，则抛出异常
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    # 清除列表中的换行符
    usage_list = [line.strip('\n') for line in usage_list]
    # 打印数据集信息
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    # 定义最大长度
    max_len = 3000000
    # 初始化一个足够大的张量来存储所有特征，大小为最大长度乘以每帧特征的数量（39 * concat_nframes）
    X = torch.empty(max_len, 39 * concat_nframes)
    # 如果模式不是测试，则初始化一个同样大小的张量来存储所有标签
    if mode != 'test':
        y = torch.empty(max_len, dtype=torch.long)
    
    # 初始化索引变量，用于跟踪当前填充到的位置
    idx = 0
    # 遍历数据列表，使用tqdm显示进度条
    for i, fname in tqdm(enumerate(usage_list)):
        # 加载当前文件的特征
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        # 获取加载的特征的长度
        cur_len = len(feat)
        # 将当前特征与上下文特征进行拼接
        feat = concat_feat(feat, concat_nframes)
        # 如果模式不是测试，则获取当前文件的标签
        if mode != 'test':
            label = torch.LongTensor(label_dict[fname])
    
        # 将拼接后的特征复制到X张量的当前索引位置
        X[idx: idx + cur_len, :] = feat
        # 如果模式不是测试，则将标签复制到y张量的当前索引位置
        if mode != 'test':
            y[idx: idx + cur_len] = label
    
        # 更新索引变量，指向下一个可填充的位置
        idx += cur_len
    
    # 截断X和y张量，只保留实际填充的数据
    X = X[:idx, :]
    if mode != 'test':
        y = y[:idx]
    
    # 打印当前数据集的信息
    print(f'[INFO] {split} set')
    print(X.shape)
    # 如果模式不是测试，则打印标签的形状
    if mode != 'test':
        print(y.shape)
        # 返回处理后的特征和标签
        return X, y
    else:
        # 如果模式是测试，则只返回特征
        return X
    # 返回的X代表数据的维度，如果不链接则为39 如果链接即为n*39 n为连接的特征总数,y为标签

## 定义数据集

In [4]:
import torch
# 导入数据集模块，用于自定义数据集
from torch.utils.data import Dataset
# 导入数据加载工具模块，用于加载数据集
from torch.utils.data import DataLoader

# 定义一个数据集类LibriDataset，继承自Dataset
# 这个LibriDataset类是自定义的PyTorch数据集，用于处理和提供音频数据及其标签（如果提供）。
# 它可以用于加载和迭代LibriSpeech数据集，其中__getitem__方法用于获取每个样本（数据和标签），而__len__方法用于返回数据集的大小。
class LibriDataset(Dataset):
    # 初始化方法，接收数据特征X和标签y（如果有的话）
    def __init__(self, X, y=None):
        # 将数据特征赋值给实例变量
        self.data = X
        # 如果标签y不是None，则将标签转换为LongTensor并赋值给实例变量
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            # 如果没有标签，则将实例变量label设置为None
            self.label = None

    # 定义获取数据集中单个元素的方法
    def __getitem__(self, idx):
        # 根据索引idx获取数据特征
        data_item = self.data[idx]
        # 如果有标签，则同时获取对应的标签
        if self.label is not None:
            label_item = self.label[idx]
            # 返回数据和标签
            return data_item, label_item
        else:
            # 如果没有标签，则只返回数据
            return data_item

    # 定义获取数据集长度的方法
    def __len__(self):
        # 返回数据集的长度
        return len(self.data)

#  &#x2728; 神经网络模型
<font color=darkred><b>***TODO***: 使用近似相同数量的参数实现2个模型，（A）一个更窄和更深（例如hidden_layers=6，hidden_dim＝1024）和（B）另一个更宽和更浅（例如 hidden_layers＝2、hidden_dim＝1700）。报告两种模型的训练/验证精度。</font></b>

<font color=darkred><b>***TODO***:  添加dropout层，并报告dropout率分别等于（A）0.25/（B）0.5/（C）0.75的训练/验证准确性。</font></b>

Dropout层在神经网络层当中是用来干什么的呢？它是一种可以用于减少神经网络过拟合的结构。
![](./pic/02.png)   
如上图我们定义的网络,一共有四个输入x_i，一个输出y。Dropout则是在每一个batch的训练当中随机减掉一些神经元，而作为编程者，我们可以设定每一层dropout（将神经元去除的的多少）的概率，在设定之后，就可以得到第一个batch进行训练的结果：  
![](./pic/03.png)   
从上图我们可以看到一些神经元之间断开了连接，因此它们被dropout了！dropout顾名思义就是被拿掉的意思，正因为我们在神经网络当中拿掉了一些神经元，所以才叫做dropout层。
在进行第一个batch的训练时，有以下步骤：
* 设定每一个神经网络层进行dropout的概率
* 根据相应的概率拿掉一部分的神经元，然后开始训练，更新没有被拿掉神经元以及权重的参数，将其保留
* 参数全部更新之后，又重新根据相应的概率拿掉一部分神经元，然后开始训练，如果新用于训练的神经元已经在第一次当中训练过，那么我们继续更新它的参数。而第二次被剪掉的神经元，同时第一次已经更新过参数的，我们保留它的权重，不做修改，直到第n次batch进行dropout时没有将其删除。

PS: 上面的两个TODO是可以改进其他部分来提高你的成绩的方法。  
如下的策略是助教给出的几个优化方式：  
● (1%) Simple baseline: 0.45797 (sample code)  
● (1%) Medium baseline: 0.69747 (concat n frames, add layers)  
● (1%) Strong baseline: 0.75028 (concat n, batchnorm, dropout, add layers)  
● (1%) Boss baseline: 0.82324 (sequence-labeling(using RNN))  
对于boss baseline，您可以参考RNN之前的课程记录

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# 定义基本的神经网络块BasicBlock，它是一个全连接层后面跟着ReLU激活函数
class BasicBlock(nn.Module):  # 继承自torch.nn.Module
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()  # 调用父类的初始化方法

        # 定义神经网络块，包含一个全连接层和一个ReLU激活函数
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),  # 输入维度到输出维度的全连接层
            nn.ReLU(),  # ReLU激活函数
        )

    def forward(self, x):  # 定义前向传播方法
        x = self.block(x)  # 将输入x通过定义的神经网络块
        return x

# 定义分类器网络Classifier，它由多个BasicBlock组成，最后是一个全连接层
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()  # 调用父类的初始化方法

        # 定义神经网络的结构，首先是一个BasicBlock，然后是多个隐藏层BasicBlock，最后是一个输出层
        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),  # 输入层到第一个隐藏层的BasicBlock
            *([BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)]),  # *[]用于解包列表，创建多个隐藏层
            nn.Linear(hidden_dim, output_dim)  # 最后的全连接层，从最后一个隐藏层到输出层
        )

    def forward(self, x):  # 定义前向传播方法
        x = self.fc(x)  # 将输入x通过定义的神经网络结构
        return x

# 这个Classifier类定义了一个简单的全连接神经网络，用于分类任务。
# 它包含一个输入层、一个或多个隐藏层（数量和大小可配置）以及一个输出层。
# 每个隐藏层都是BasicBlock的实例，输出层是一个线性层。

## 超参数定义

<font color=darkred><b>***TODO***:  可以考虑进一步优化超参数来提高准确率。</font></b>

In [6]:
# data prarameters
# 用于数据处理时的参数
concat_nframes = 1              # 要连接的帧数,n必须为奇数（总共2k+1=n帧）
train_ratio = 0.8               # 用于训练的数据比率，其余数据将用于验证
# training parameters
# 训练过程中的参数
seed = 0                        # 随机种子
batch_size = 512                # 批次数目
num_epoch = 5                   # 训练epoch数
learning_rate = 0.0001          # 学习率
model_path = './model.ckpt'     # 选择保存检查点的路径（即下文调用保存模型函数的保存位置）
# model parameters
# 模型参数
input_dim = 39 * concat_nframes # 模型的输入维度，不应更改该值，这个值由上面的拼接函数决定
hidden_layers = 1               # hidden_layer层的数量
hidden_dim = 256                # 隐藏维度

## 准备数据与模型

In [7]:
# 引入gc模块进行垃圾回收
import gc

# 预处理数据
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# 将数据导入
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# 删除原始数据以节省内存
del train_X, train_y, val_X, val_y
gc.collect()

# 利用dataloader加载数据
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for train: 3428


3428it [00:03, 1004.06it/s]


[INFO] train set
torch.Size([2116368, 39])
torch.Size([2116368])
[Dataset] - # phone classes: 41, number of utterances for val: 858


858it [00:00, 971.03it/s]


[INFO] val set
torch.Size([527790, 39])
torch.Size([527790])


In [8]:
# 检查当前是否有可用的GPU 否则使用CPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cuda:0


In [9]:
import numpy as np

# 固定随机种子
def same_seeds(seed): # 固定随机种子（CPU）
    torch.manual_seed(seed) # 固定随机种子（GPU)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed) # 为当前GPU设置
        torch.cuda.manual_seed_all(seed)  # 为所有GPU设置
    np.random.seed(seed)  # 保证后续使用random函数时，产生固定的随机数
    torch.backends.cudnn.benchmark = False # GPU、网络结构固定，可设置为True
    torch.backends.cudnn.deterministic = True # 固定网络结构

In [10]:
# 固定随机种子
same_seeds(seed)

# 创建模型、定义损失函数和优化器
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

## 训练模型

In [11]:
# 定义最佳准确率best_acc初始为0.0
best_acc = 0.0

# 循环遍历指定的训练周期num_epoch
for epoch in range(num_epoch):
    # 初始化训练集和验证集的准确率和损失
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # 训练部分
    # 设置模型到训练模式
    model.train()
    # 遍历train_loader中的每个批次
    for i, batch in enumerate(tqdm(train_loader)):
        # 获取当前批次的数据和标签
        features, labels = batch
        # 将特征和标签移动到指定的device上
        features = features.to(device)
        labels = labels.to(device)
        
        # 重置优化器的梯度
        optimizer.zero_grad() 
        # 计算模型输出
        outputs = model(features) 
        # 计算损失
        loss = criterion(outputs, labels)
        # 执行反向传播
        loss.backward() 
        # 执行优化器步骤
        optimizer.step() 
        
        # 获取概率最高的类的索引
        _, train_pred = torch.max(outputs, 1) 
        # 计算训练集的准确率
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        # 累加损失
        train_loss += loss.item()
    
    # 验证部分
    # 如果存在验证集val_set
    if len(val_set) > 0:
        # 设置模型到评估模式
        model.eval()
        # 遍历val_loader中的每个批次
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                # 获取当前批次的数据和标签
                features, labels = batch
                # 将特征和标签移动到指定的device上
                features = features.to(device)
                labels = labels.to(device)
                # 计算模型输出
                outputs = model(features)
                # 计算损失
                loss = criterion(outputs, labels) 
                # 获取概率最高的类的索引
                _, val_pred = torch.max(outputs, 1) 
                # 计算验证集的准确率
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item()
                # 累加损失
                val_loss += loss.item()

            # 打印每个周期的训练和验证准确率及损失
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # 如果验证准确率超过了当前最佳准确率
            if val_acc > best_acc:
                # 更新最佳准确率
                best_acc = val_acc
                # 保存模型的状态到指定的路径
                torch.save(model.state_dict(), model_path)
                # 打印保存模型的信息
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        # 如果没有验证集，则只打印训练准确率及损失
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

    # 如果结束验证，则保存最后一个epoch得到的模型
    if len(val_set) == 0:
        torch.save(model.state_dict(), model_path)
        print('saving model at last epoch')

100%|█████████████████████████████████████████████████████████████████████████████| 1031/1031 [00:03<00:00, 270.45it/s]


[001/005] Train Acc: 0.421913 Loss: 2.086644 | Val Acc: 0.440581 loss: 1.971926
saving model with acc 0.441


100%|█████████████████████████████████████████████████████████████████████████████| 1031/1031 [00:04<00:00, 210.34it/s]


[002/005] Train Acc: 0.449024 Loss: 1.934470 | Val Acc: 0.449734 loss: 1.926857
saving model with acc 0.450


100%|█████████████████████████████████████████████████████████████████████████████| 1031/1031 [00:04<00:00, 212.63it/s]


[003/005] Train Acc: 0.455070 Loss: 1.904343 | Val Acc: 0.453828 loss: 1.908872
saving model with acc 0.454


100%|█████████████████████████████████████████████████████████████████████████████| 1031/1031 [00:04<00:00, 243.26it/s]


[004/005] Train Acc: 0.458388 Loss: 1.887978 | Val Acc: 0.456020 loss: 1.896504
saving model with acc 0.456


100%|█████████████████████████████████████████████████████████████████████████████| 1031/1031 [00:04<00:00, 225.30it/s]

[005/005] Train Acc: 0.460761 Loss: 1.876417 | Val Acc: 0.457811 loss: 1.889774
saving model with acc 0.458


In [12]:
del train_loader, val_loader
gc.collect()

0

## 测试
创建测试数据集，并从保存的检查点加载模型。

In [13]:
# 载入数据
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 1078


1078it [00:01, 1012.06it/s]

[INFO] test set
torch.Size([646268, 39])


In [14]:
# 加载已经训练好的模型
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [15]:
# 定义测试集的准确率为0.0
test_acc = 0.0
# 定义测试集的长度为0
test_lengths = 0
# 初始化一个空的整数数组pred，用于存储预测的类别索引
pred = np.array([], dtype=np.int32)

# 将模型设置为评估模式
model.eval()

# 使用torch.no_grad上下文管理器，确保在测试过程中不计算梯度
with torch.no_grad():
    # 遍历test_loader中的每个批次
    for i, batch in enumerate(tqdm(test_loader)):
        # 获取当前批次的数据
        features = batch
        # 将特征移动到指定的device上
        features = features.to(device)

        # 计算模型在当前批次上的输出
        outputs = model(features)

        # 使用torch.max找到每个样本的概率最高的类别索引
        _, test_pred = torch.max(outputs, 1)

        # 将当前批次的预测结果与之前的预测结果合并
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)

100%|█████████████████████████████████████████████████████████████████████████████| 1263/1263 [00:03<00:00, 331.29it/s]


将预测结果写入CSV文件。

In [16]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))

# 参考文献：  
[一文入门dropout层](https://www.cnblogs.com/geeksongs/p/13446980.html)  
李宏毅机器学习2022在线课程

# 贡献者  
潘笃驿(panduyi_azula@foxmail.com)